## Imports

In [2]:
import sys
sys.path.append('../')

In [24]:
from munch import Munch
from PIL import Image

In [25]:
from core.data_loader import get_test_loader, InputFetcher, get_test_transform

In [3]:
%load_ext autoreload
%autoreload 2

## Objective

* Create a `predict` function that accepts 1 source image and 1 reference image

## Get shape of data passed to `utils.translate_using_reference`

In [15]:
args = Munch(src_dir='../assets/representative/celeba_hq/src',
            ref_dir='../assets/representative/celeba_hq/ref',
            img_size=256,
            val_batch_size=32,
            num_workers=4,
            latent_dim=16)

In [11]:
loaders = Munch(src=get_test_loader(root=args.src_dir,
                                            img_size=args.img_size,
                                            batch_size=args.val_batch_size,
                                            shuffle=False,
                                            num_workers=args.num_workers),
                        ref=get_test_loader(root=args.ref_dir,
                                            img_size=args.img_size,
                                            batch_size=args.val_batch_size,
                                            shuffle=False,
                                            num_workers=args.num_workers))

Preparing DataLoader for the generation phase...
Preparing DataLoader for the generation phase...


In [16]:
src = next(InputFetcher(loaders.src, None, args.latent_dim, 'test'))
ref = next(InputFetcher(loaders.ref, None, args.latent_dim, 'test'))

In [18]:
src.x.shape

torch.Size([10, 3, 256, 256])

In [21]:
ref.x.shape

torch.Size([32, 3, 256, 256])

In [22]:
ref.y.shape

torch.Size([32])

In [23]:
ref.y

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')